In [1]:
import os, sys
from pathlib import Path
import collections
import numpy as np
import scipy
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"]="4" 
import math
import torch
import torch.nn as nn
import time
sys.path.append("../..")
#sys.path.append("../lib")
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt
from lib import *
basepath = Path.home()/'Samuel_Susoy_mating/'
net = 'synaptic_partners_v1.csv'
neuron = 'transmitters_and_annotations.csv'
calcium_folder = 'normalized_activity_id_not_filtered/'

In [2]:
mdata = mating_data(basepath, net, neuron, calcium_folder)
subnet_df = mdata.extract_subnet()
subneuron_df = mdata.extract_subneuron()
connectome = Connectome(subnet_df, subneuron_df, mdata.neurons)
network = Network(connectome, 'cuda')
network.cuda()
encoder_net = encoder.Encoder(len(mdata.behavior_list)*2,len(connectome.male_sensory_neuron_list))
encoder_net.cuda()
batch_size = 100
x_in = torch.from_numpy(np.random.uniform(size = (1,len(connectome.neuron_list)))).to(torch.float32).cuda()
c_in = torch.from_numpy(np.random.uniform(size = (1,len(connectome.neuron_list)))).to(torch.float32).cuda()
stim_features_np = mdata.behavior_matrix_list[0]
miss_behavior = np.zeros(shape = mdata.behavior_matrix_list[0].shape)
miss_behavior[np.isnan(mdata.behavior_matrix_list[0])] = 1
missing_stim = torch.from_numpy(miss_behavior).to(torch.float32).cuda()
stim_features_np[np.isnan(stim_features_np)] = 0
stim_features = torch.from_numpy(stim_features_np).to(torch.float32).cuda()
target = torch.from_numpy(mdata.activity_matrix_list[0]).to(torch.float32).cuda()
miss_activity = np.zeros(shape = mdata.activity_matrix_list[0].shape)
miss_activity[mdata.activity_matrix_list[0] == 0] = 1
missing_target = torch.from_numpy(miss_activity).to(torch.float32).cuda()
sensory_neuron_indices = [n.index for n in connectome.male_sensory_neuron_list]
encoder_input = torch.cat((stim_features, missing_stim), dim = 1)

TypeError: 'torch.device' object is not callable

In [ ]:
def train(encoder_input,x_init, c_init, target, missing_target, learning_rate, encoder_net, network, epochs, batch_size):
    print("training")
    loss_list,activity_list = [],[]
    criterion = network.loss
    params = list(encoder_net.parameters()) + list(network.parameters())
    optimizer = torch.optim.Adam(params, lr = learning_rate)
    for epoch in range(epochs):
        loss_sum = 0
        T = target.shape[0]
        batch_num = int(T/batch_size)
        start_indices = np.random.choice(batch_num, batch_num) 
        x,c,s = x_init, c_init, torch.zeros(x_init.shape[1])
        activity_trace = torch.zeros((target.shape[0], x_init.shape[1]))
        for index in start_indices:
            optimizer.zero_grad()
            seq_length = batch_size
            if index == batch_num:
                seq_length = target.shape[1]%batch_size
            for i in range(seq_length):
                sensory_input = encoder_net.forward(encoder_input[index*batch_size + i,:])
                for k, neuron_index in enumerate(sensory_neuron_indices):
                    s[neuron_index] = sensory_input[k]
                x,c, = network.forward(x,c,s)
                activity_trace[index*batch_size + i,:] = x
                loss = criterion(x,target[index*batch_size + i,:], missing_target[index*batch_size + i,:])
                loss.backward(retain_graph = True)
                loss_sum += loss.item()
            optimizer.step()
        epoch_loss = loss_sum/target.shape[0]
        print("epoch number: ", epoch, "Loss: ", epoch_loss)
        loss_list.append(epoch_loss)
        if epoch%50 == 0:
            activity_list.append(activity_trace)
    print("Done training")
    return loss_list, activity_list

In [ ]:
start_time = time.time()
loss_list,activity_list = train(encoder_input, x_in, c_in,target, missing_target, 1e-3, encoder_net, network, epochs = 1, batch_size = batch_size)
end_time = time.time()
print("total training time: ", end_time-start_time)
torch.save(network.state_dict(), Path.home()/"worm_mating/experiments/test")

In [ ]:
plt.plot(loss_list)
print(loss_list)

In [ ]:
statedict = torch.load(Path.home()/"worm_mating/experiments/test")
network.load_state_dict(statedict)
x_init = torch.from_numpy(np.zeros(shape = (1,len(connectome.neuron_list)))).to(torch.float32)
c_init = torch.from_numpy(np.zeros(shape = (1,len(connectome.neuron_list)))).to(torch.float32)
print(missing_target.shape)
sense_in = encoder_net.predict(stim_features, missing_stim, missing_target, target)
print(sense_in.shape)
s = torch.zeros(stim_features.shape[0],x_init.shape[1])
for k, neuron_index in enumerate(sensory_neuron_indices):
    s[:,neuron_index] = sense_in[:,k]
x_out, c_out, loss = network.predict(x_init,c_init,s,target, missing_target)


In [ ]:
mating_activity = x_out[-54:,:].data.numpy()
np_target = target.data.numpy()

In [ ]:
for i in range(mating_activity.shape[0]):
    f,axes = plt.subplots(1,2,sharex=True,figsize=(20,7))
    axes[0].set_title("Predicted : " + connectome.neuron_names[i])
    axes[1].set_title("Ground Truth : " + connectome.neuron_names[i])
    axes[0].plot(mating_activity[i,:])
    axes[1].plot(np_target[:,i])
    